In [3]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [4]:
spark = SparkSession \
        .builder \
        .master("local[3]") \
        .appName("LogFileDome") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/26 00:16:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
file_df = spark.read.text("../data/apache_logs.txt")


In [10]:
file_df.printSchema()

root
 |-- value: string (nullable = true)



In [12]:
log_reg = r'^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+) (\S+)" (\d{3}) (\S+) "(\S+)" "([^"]*)'

logs_df = file_df.select(regexp_extract('value', log_reg, 1).alias('ip'),
                         regexp_extract('value', log_reg, 4).alias('date'),
                         regexp_extract('value', log_reg, 6).alias('request'),
                         regexp_extract('value', log_reg, 10).alias('referrer'),
                         )

In [13]:
logs_df.printSchema()

root
 |-- ip: string (nullable = true)
 |-- date: string (nullable = true)
 |-- request: string (nullable = true)
 |-- referrer: string (nullable = true)



In [14]:
logs_df \
    .where("trim(referrer) != '-'") \
    .withColumn("referrer", substring_index("referrer", "/", 3)) \
    .groupBy("referrer") \
    .count() \
    .show(10, truncate=False)

+---------------------------------+-----+
|referrer                         |count|
+---------------------------------+-----+
|http://ijavascript.cn            |1    |
|http://www.google.co.tz          |1    |
|http://www.google.ca             |6    |
|https://www.google.hr            |2    |
|https://www.google.ch            |1    |
|http://www.google.ru             |6    |
|http://www.raspberrypi-spanish.es|1    |
|http://semicomplete.com          |2001 |
|http://manpages.ubuntu.com       |2    |
|http://kufli.blogspot.fr         |1    |
+---------------------------------+-----+
only showing top 10 rows

